In [2]:
import random
import time

# Dimensão do tabuleiro (3x3)
N = 3

# Classe que representa um estado do puzzle
class EstadoPuzzle:
    def __init__(self, tabuleiro, x, y, profundidade, movimento=""):
        self.tabuleiro = [linha[:] for linha in tabuleiro]  # Cópia do tabuleiro
        self.x = x
        self.y = y
        self.profundidade = profundidade
        self.movimento = movimento  # Nome do movimento realizado

# Movimentos possíveis e seus nomes
movimentos_possiveis = [(0, -1), (0, 1), (-1, 0), (1, 0)]
nome_movimentos = ["Esquerda", "Direita", "Cima", "Baixo"]

# Estado objetivo do puzzle
estado_objetivo = [[1, 2, 3], [4, 5, 6], [7, 8, 0]]

# Verifica se um estado é o objetivo
def verificar_objetivo(tabuleiro):
    return tabuleiro == estado_objetivo

# Verifica se um movimento é válido
def movimento_valido(x, y):
    return 0 <= x < N and 0 <= y < N

# Conta inversões para verificar se o tabuleiro é solucionável
def contar_inversoes(tabuleiro):
    lista_plana = [num for linha in tabuleiro for num in linha if num != 0]
    inversoes = sum(1 for i in range(len(lista_plana)) for j in range(i + 1, len(lista_plana))
                    if lista_plana[i] > lista_plana[j])
    return inversoes

def eh_solucionavel(tabuleiro):
    return contar_inversoes(tabuleiro) % 2 == 0

# Gera um tabuleiro aleatório e solucionável
def gerar_tabuleiro_aleatorio():
    while True:
        numeros = list(range(N * N))
        random.shuffle(numeros)
        tabuleiro = [numeros[i:i+N] for i in range(0, N * N, N)]
        if eh_solucionavel(tabuleiro):
            return tabuleiro

# Função para permitir que o usuário insira um tabuleiro inicial
def criar_tabuleiro_definido():
    print("Insira o tabuleiro linha por linha, inserindo os números separados por espaço.")
    print("Exemplo para a primeira linha: 1 2 3")
    tabuleiro = []
    for i in range(N):
        linha = list(map(int, input().split()))
        tabuleiro.append(linha)
    return tabuleiro

# Função para salvar os movimentos em um arquivo
def salvar_solucao_arquivo(movimentos, caminho, total_passos, tempo_decorrido, inicio):
    with open(caminho, "w") as arquivo:
        arquivo.write(f"\nTotal de passos: {total_passos}\n")
        arquivo.write(f"Tempo de processamento: {tempo_decorrido:.4f} segundos\n")
        arquivo.write(f"Tabuleiro inicial:\n")
        
        for linha in inicio:
            arquivo.write(" ".join(map(str, linha)) + "\n")
            
        arquivo.write("--------\n")
        arquivo.write("Sequência de movimentos:\n")
        for movimento, tabuleiro in movimentos:
            arquivo.write(f"{movimento}\n")
            for linha in tabuleiro:
                arquivo.write(" ".join(map(str, linha)) + "\n")
            arquivo.write("--------\n")

# Algoritmo de Busca de Profundidade Iterativa
def resolver_puzzle_ids(tabuleiro, x, y):
    tempo_inicio = time.time()
    profundidade_maxima = 0
    tabuleiro_inicial = [linha[:] for linha in tabuleiro]  # Guardar cópia do tabuleiro inicial

    while True:
        movimentos = []  # Lista de tuplas (movimento, tabuleiro)
        estados_visitados = []
        if busca_dfs_limitada(tabuleiro, x, y, profundidade_maxima, movimentos, estados_visitados):
            tempo_execucao = time.time() - tempo_inicio
            salvar_solucao_arquivo(movimentos, "solucaoIDS.txt", len(estados_visitados), tempo_execucao, tabuleiro_inicial)
            return
        profundidade_maxima += 1

# Busca em Profundidade com Limite de Profundidade
def busca_dfs_limitada(tabuleiro, x, y, limite, movimentos, estados_visitados):
    pilha = [EstadoPuzzle(tabuleiro, x, y, 0)]
    visitados = set()
    visitados.add(tuple(map(tuple, tabuleiro)))

    while pilha:
        atual = pilha.pop()
        if atual.profundidade > limite:
            continue

        estados_visitados.append(atual.tabuleiro)

        if verificar_objetivo(atual.tabuleiro):
            return True

        for i, (dx, dy) in enumerate(movimentos_possiveis):  # Corrigido para usar movimentos_possiveis
            novo_x, novo_y = atual.x + dx, atual.y + dy
            if movimento_valido(novo_x, novo_y):
                novo_tabuleiro = [linha[:] for linha in atual.tabuleiro]
                novo_tabuleiro[atual.x][atual.y], novo_tabuleiro[novo_x][novo_y] = novo_tabuleiro[novo_x][novo_y], novo_tabuleiro[atual.x][atual.y]
                tupla_tabuleiro = tuple(map(tuple, novo_tabuleiro))
                if tupla_tabuleiro not in visitados:
                    visitados.add(tupla_tabuleiro)
                    pilha.append(EstadoPuzzle(novo_tabuleiro, novo_x, novo_y, atual.profundidade + 1, nome_movimentos[i]))
                    movimentos.append((nome_movimentos[i], novo_tabuleiro))  # Adiciona o movimento corretamente
    return False

# Executa o programa
def main():
    print("Escolha uma opção:")
    print("1. Definir um tabuleiro manualmente")
    print("2. Tabuleiro aleatório solucionável")

    escolha = input("Escolha uma opção:\n1 - Inserir tabuleiro manualmente\n2 - Gerar tabuleiro aleatório\nOpção: ")


    if escolha == "1":
        tabuleiro = criar_tabuleiro_definido()
    else:
        tabuleiro = gerar_tabuleiro_aleatorio()
    
    # Encontra a posição do espaço vazio (0)
    for i in range(N):
        for j in range(N):
            if tabuleiro[i][j] == 0:
                resolver_puzzle_ids(tabuleiro, i, j)
                return

if __name__ == "__main__":
    main()


Escolha uma opção:
1. Definir um tabuleiro manualmente
2. Tabuleiro aleatório solucionável


Insira o tabuleiro linha por linha, inserindo os números separados por espaço.
Exemplo para a primeira linha: 1 2 3
